In [1]:
#This attempt is merely that as some of the suggested implementations of Noterdame or Prochaska were not finished or included
# Initialising
#various imported libraries, some may be unused
#filenames may be irrelevant depending on who is accessing or what is available
import matplotlib.transforms as mtransforms
%matplotlib inline
from __future__ import print_function
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib
import math as mt
import numpy as np
import pickle
import os
from scipy.optimize import curve_fit
from astropy.io import fits
import pandas as pd
from astropy.utils.data import download_file
import statistics
from scipy.stats import powerlaw
from scipy.stats import iqr
import time
from astropy import constants as const
np.set_printoptions(threshold=np.nan)
from astropy.table import Table


In [631]:
#FITS file reader function 
def rSDSSFITS(FITSfile):
    with fits.open(FITSfile) as hdulist:
        #allows for easy access of different headers from the FITS file. For DR14, the lite versions were sufficient. 
        (h0,d0,h1,d1,d2,h2)=(hdulist[0].header,hdulist[0].data,hdulist[1].header,hdulist[1].data,hdulist[2].data,hdulist[2].header)
        #flux array in angstrom
        lw=d1['loglam']
        #wavelength array in angstrom
        wl=np.power(10.0,lw)
        #the inverse variance array which corresponds to each wavelength value (for each pixel)
        invaris=np.array(d1['ivar'])
        #assignszeroes in ivar array as -inf
        sig=np.empty((len(invaris)))
        for j in range(0,len(invaris)):
            sig[j]=np.reciprocal(np.sqrt(invaris[j])) if (invaris[j]>0.0) else -np.inf
    #Returned variables 
    return {'wl':wl,'sig':sig,'lw':lw,'fl':d1['flux'],'er':d1[2],'invaris':invaris,'h0':h0,'d0':d0,'d1':d1,'h1':h1,'h2':h2,'d2':d2}


In [633]:
#imports input table of observations for processing
#accesses the input table and specifes the filetype 
dat = Table.read('nondlas', format='csv')
df = dat.to_pandas()
#displays the input table
df

,col1,plate_1a,plate mjd fiber_1,mjd_1a,fiberid_1,run2d,specobj_id,ra_1a,dec_1a,z,...,dlas/0/column_density_bias_adjust,dlas/0/type,dlas/0/spectrum,dlas/0/rest,dlas/0/z_dla,Plate_2a,plate mjd fiber,MJD_2a,Fiber_2,col6
0,129081.0,5194.0,5194 56062 356,56062.0,356.0,'v5_10_0','5848022074379968512','16:46:46.64','+30:55:56.18',2.25754,...,0.078393,DLA,3885.974609,1195.911564,2.196570,5194.0,5194 56062 356,56062.0,356.0,05194-56062-00356
1,118701.0,7376.0,7376 56749 860,56749.0,860.0,'v5_10_0','8304874221101621248','08:20:55.64','+54:36:29.94',2.29410,...,0.038024,DLA,3921.028564,1190.969895,2.225405,7376.0,7376 56749 860,56749.0,860.0,07376-56749-00860
2,11333.0,4717.0,4717 55742 124,55742.0,124.0,'v5_10_0','5310904041772916736','15:09:53.08','+34:32:29.97',3.28796,...,0.043050,DLA,5116.819824,1194.538149,3.209053,4717.0,4717 55742 124,55742.0,124.0,04717-55742-00124
3,83972.0,5329.0,5329 55946 235,55946.0,235.0,'v5_10_0','5999985299630825472','10:04:44.14','+08:08:43.07',2.43594,...,0.057124,DLA,4533.148926,1314.384084,2.728930,5329.0,5329 55946 235,55946.0,235.0,05329-55946-00235
4,12319.0,5344.0,5344 55924 314,55924.0,314.0,'v5_10_0','6016895513219014656','10:31:19.33','+08:20:00.51',1.51504,...,NaN,NaN,NaN,NaN,NaN,5344.0,5344 55924 314,55924.0,314.0,05344-55924-00314
5,103653.0,5298.0,5298 55979 522,55979.0,522.0,'v5_10_0','5965161293031645184','09:02:21.50','+13:25:14.38',2.35869,...,0.071580,DLA,4215.995117,1256.188752,2.468042,5298.0,5298 55979 522,55979.0,522.0,05298-55979-00522
6,47354.0,4661.0,4661 55614 278,55614.0,278.0,'v5_10_0','5247895976039915520','01:37:10.07','+13:36:12.70',2.73020,...,NaN,NaN,NaN,NaN,NaN,4661.0,4661 55614 278,55614.0,278.0,04661-55614-00278
7,97812.0,5068.0,5068 55749 482,55749.0,482.0,'v5_10_0','5706193315482804224','22:07:54.33','+07:36:03.94',2.39056,...,0.098732,DLA,3661.844971,1082.728995,2.012203,5068.0,5068 55749 482,55749.0,482.0,05068-55749-00482
8,123139.0,6315.0,6315 56181 896,56181.0,896.0,'v5_10_0','7110304306016788480','16:55:53.79','+49:30:53.16',2.27008,...,NaN,NaN,NaN,NaN,NaN,6315.0,6315 56181 896,56181.0,896.0,06315-56181-00896
9,132072.0,4782.0,4782 55654 718,55654.0,718.0,'v5_10_0','5384250811718017024','14:15:58.40','+05:19:57.96',2.22803,...,0.040419,DLA,4161.021973,1285.339384,2.422822,4782.0,4782 55654 718,55654.0,718.0,04782-55654-00718


In [628]:
#initialises empty variables for Zmin and Zmax
ZMIN=[]
ZMAX=[]
#Whether zmin is less than zman (true or false, 1 or 0)
ZVS=[]
#the plate-mjd-fiber of the observation
Bname=[]
#speed of light (km/s)
c=299792.458
#Hydrogen alpha line
a=1215.6701
#zmax separation offset
zmo=5000.0
#the range of concern in km/s
v=5000.0
#speed of light in km/s
c=299792.458
#lyman beta,O6 line (Angstrom)
lybO6=1025.7
#iteratively works through the input list
for t in range(0,27393):
    #acts as a progress bar displaying when 250 files have been processed
    if t%250==0:
        print(t)
    #forms a string using the current row of the input table which can be used to access the relevant observation
    FITSfile='spec'+'-'+str(int(df.iloc[t]['plate']))+'-'+str(int(df.iloc[t]['mjd']))+'-'+str(int(df.iloc[t]['fiberid'])).zfill(4)+'.fits' 
    #uses the string to access the file with the function
    s=rSDSSFITS(FITSfile)
    siggie=s['sig']
    invar=s['invaris']
    #the quasar emission redshift from input table
    zuse=df.iloc[t]['Zem']
    #conveniant for frame conversion
    z1=zuse+1
    x=s['wl']
    y=np.array(s['fl'])
    SNR=y/siggie
    #converts the km/s value to Angstrom 
    lyboffset=(((v/c+1)*lybO6)*(z1))-lybO6*z1
    #range shorthand
    lwr=(lybO6*z1)-lyboffset
    upr=(lybO6*z1)+lyboffset
    wind=0  
    i=0
    A=0
    SNRval=5
    while wind <SNRval:
        if (x[i]>=lwr) & (x[i]<=upr):
            SNRval=15
        else:
            SNRval=5
        wind=np.median(SNR[i:i+33])
        i=i+1  
        if i==len(SNR)-17:
            A=1
            break   
    if A==1:
        zmin=-np.inf  
        zmax=-np.inf
        zvs=0  
    else:
        zmin=(((x[i+17])/a)-1)
        zmax=zuse-(zmo/c)*(1.0+zuse)
        zvs=zmin<zmax
    ZMIN.append(zmin)
    ZMAX.append(zmax)
    ZVS.append(zvs*1)
    Bname.append(df.iloc[t]['plate mjd fiber'])
    print(zmin,zmax,zuse,zvs*1)   
f = open('NONDLAZVAL1.pckl', 'wb')
pickle.dump(ZMIN, f)
pickle.dump(ZMAX, f)
pickle.dump(ZVS, f)    
pickle.dump(Bname, f)  

0
2.734944980138937 1.9023191141917086 1.951545596 0
2.719496485158679 1.9024105454188736 1.951638578 0
1.9470275539803108 1.903340338125546 1.952584141 0
2.7263555655179394 1.9037473763161115 1.952998083 0
-inf -inf 1.953221859 0
1.9983640169935906 1.9053756156106905 1.954653939 0
2.1274054618492917 1.9055510539854665 1.954832353 0
2.350304285754622 1.9069020948659672 1.956206309 0
-inf -inf 1.956443078 0
2.751320497281705 1.9086586753546175 1.957992683 0
2.361895075206361 1.9096580311950508 1.959008989 0
2.3937843564374495 1.9117945849251743 1.961181781 0
2.719496485158679 1.913235876063364 1.962647518 0
2.3758584469431714 1.9158338761623048 1.965289583 0
2.701554911078466 1.91707322447005 1.966549952 0
2.764300815837084 1.9178661515660085 1.967356328 0
2.362670271382117 1.9179537134219093 1.967445375 0
2.747004703028601 1.9180227081956722 1.96751554 0
2.7599697586540954 1.9185268317137252 1.968028214 0
1.937542848024723 1.920566241117053 1.970102214 0
2.0247138588164253 1.9210485063

-inf -inf 2.067787689 0
2.0980209077435976 2.0172632882606987 2.068439348 0
2.0745684118346537 2.017777132898019 2.068961908 0
2.057625153865757 2.018374706402883 2.069569617 0
2.131729891707462 2.0188818034860447 2.070085315 0
2.100874272320056 2.0195343348460444 2.070748914 0
2.073861296359308 2.019716403759685 2.070934071 0
2.1440121320486125 2.020739270824243 2.071974287 0
2.066791949058075 2.022674367484888 2.073942205 0
2.865308877537993 2.0242825430815574 2.075577657 0
2.0682049750406173 2.0243795005772114 2.075676259 0
2.073861296359308 2.024611123992724 2.075911811 0
2.1037322559410856 2.024966011711878 2.076272718 0
2.001125803990737 2.0264119224958623 2.077743153 1
2.1216505339704206 2.0267525195993668 2.078089527 0
2.0275013518007476 2.0278377184492027 2.079193132 1
2.0254095270943573 2.0283965067924967 2.079761398 1
2.9208864267791483 2.028429601470837 2.079795054 0
2.8760033715150186 2.029077863247602 2.080454311 0
2.092318395979201 2.0294135004759712 2.080795641 0
2.1281

2.176759752257726 2.077777002005379 2.12997944 0
2.1173401620739867 2.078094205883375 2.130302024 0
2.181884080138703 2.0783372899144106 2.130549231 0
2.1599799683010215 2.078537651563409 2.130752991 0
-inf -inf 2.130966142 0
1.9907789433246528 2.0789891811319214 2.131212179 1
2.0261072036525367 2.079184719587514 2.131411034 1
2.9380773057026324 2.0792272600550197 2.131454296 0
2.9726872041857817 2.0792745843830596 2.131502423 0
2.1080237500031873 2.07934690278449 2.131575968 0
2.078111620676325 2.0796534041880665 2.131887668 1
2.1101710032444245 2.0806826825107287 2.132934404 0
2.131729891707462 2.0811845296388256 2.133444763 0
2.090183594075502 2.0814735436475957 2.133738679 0
2.1037322559410856 2.0816124703007923 2.133879962 0
2.090894525283319 2.082374261495427 2.134654674 0
2.111603911201423 2.0824618558009464 2.134743754 0
2.1541635870465594 2.0825255249038666 2.134808503 0
2.184814763504712 2.082660316688915 2.134945581 0
2.014283855348071 2.0827071552559873 2.134993214 1
2.1230

2.149083642159065 2.115534650314994 2.168377499 0
2.9910232045067158 2.115636415270905 2.16848099 0
2.1563443785679395 2.1157616186851276 2.168608317 0
2.124527596253807 2.1159415242908195 2.168791274 0
2.1375032957646156 2.11608613847728 2.168938341 0
2.1628905688645506 2.1162042049419125 2.16905841 0
3.027951060052188 2.1163087625317223 2.169164741 0
2.990105018777298 2.117144021630966 2.170014167 0
2.114471534567643 2.1173844526597985 2.170258676 1
2.172373065711865 2.117556449408291 2.17043359 0
-inf -inf 2.170648446 0
2.0583300602328296 2.1178521450886594 2.170734301 1
2.0973067632873836 2.117877855020318 2.170760447 1
2.1030183123128965 2.1181069630986777 2.170993441 1
2.9910232045067158 2.1183437302705013 2.171234224 0
2.1658055876446247 2.118369078339739 2.171260002 0
2.1295677771923893 2.1183764011371484 2.171267449 0
2.1375032957646156 2.118664631438447 2.171560568 0
3.0214639131948706 2.118693737763586 2.171590168 0
2.020537238384513 2.11876021326691 2.171657771 1
2.15416358

2.1936266955771964 2.1415328938903495 2.194816701 0
2.111603911201423 2.1419626576309243 2.195253754 1
2.1701830361013443 2.142005547177668 2.195297371 0
2.1216505339704206 2.1420733118158646 2.195366285 1
2.017061909415227 2.1420967660073247 2.195390137 1
3.060543440198126 2.14236742828143 2.19566539 0
-inf -inf 2.195678565 0
2.0973067632873836 2.142527122690942 2.195827793 1
1.9402500097991222 2.142592258909981 2.195894034 1
2.157070371877617 2.1426087954326114 2.195910851 0
2.1914218046928395 2.1426607551395938 2.195963692 0
1.9749816100653046 2.142690119095043 2.195993554 1
2.1166232060254258 2.1428338492752097 2.196139722 1
2.2328397732169485 2.1429305520905193 2.196238065 0
2.196570031108769 2.143053194934787 2.196362788 0
2.181884080138703 2.1432390073545027 2.196551752 0
2.173102874754014 2.143244589672334 2.196557429 0
2.143288950331282 2.1433188619341723 2.196632961 1
2.1259653240834004 2.143358937213937 2.196673716 1
2.1585249692612947 2.1433767097720646 2.19669179 0
2.16507

...

2.380526091895737 2.3511368926073297 2.407975811 0
2.395348003438597 2.3576980345202565 2.414648237 0
2.3642194587654783 2.3662320277341564 2.423326976 1
2.3719744329434027 2.3720022072280718 2.429195024 1
2.4133771385376264 2.381874508288546 2.43923477 0
2.433873245093385 2.3944355543165137 2.452008865 0
2.368869430851861 2.3985305517166666 2.456173318 1
2.4086641064514542 2.4004963083508875 2.458172416 0
2.439412483673613 2.403649046490648 2.461378628 0
-inf -inf 2.468077137 0
-inf -inf 2.484850983 0
2.4497237977793067 2.4298282173430406 2.488001826 0
2.173102874754014 2.444912749310949 2.503342208 1
2.52926454859649 2.494253142694378 2.553519468 0
2.5090082307023507 2.4947004046462995 2.553974316 0
2.569309255086968 2.5245517592767186 2.584331982 0
2.4888663847823516 2.526593665334085 2.586408521 1
2.551274898459294 2.5277902007941866 2.587625351 0
2.5276390465637015 2.527941854581379 2.587779577 1
2.575066994558186 2.529718992395481 2.589586857 0
2.543922986127157 2.550600559358854

2.0527008506974878 2.005234920432396 2.056206966 0
2.909166504907458 2.0337801483886335 2.085236352 0
2.1023029628885337 2.054555667046356 2.106364246 0
2.147633663819958 2.096854868682996 2.149380888 0
2.131007914958281 2.112484080293081 2.165275188 0
2.1665355975147986 2.130023429890004 2.183112024 0
-inf -inf 2.190466936 0
-inf -inf 2.190796793 0
2.235819257792986 2.182621460215666 2.236602174 0
2.2291204381975214 2.1835628069534327 2.237559487 0
-inf -inf 2.238349547 0
2.235819257792986 2.193868801108376 2.248040282 0
3.134131246703978 2.2055616457736376 2.25993145 0
2.226147179290253 2.21128646561646 2.265753369 0
-inf -inf 2.274560002 0
2.280834257069414 2.2295079789756818 2.284283939 0
-inf -inf 2.295389073 0
2.371198835111598 2.2444419230772246 2.299471179 0
2.2642557027935664 2.2630716467465914 2.318416883 0
-inf -inf 2.319414921 0
2.25975012605702 2.271378369710567 2.326864497 1
-inf -inf 2.350121038 0
2.340289996297618 2.294375833763111 2.350252023 0
-inf -inf 2.362730258 0


2.6123041252618617 2.5472831285848967 2.6074489 0
2.623967815286606 2.553763236915321 2.614038918 0
2.5652027236357133 2.567157032605311 2.627659887 1
2.6650869517098017 2.576309285464681 2.636967372 0
2.640693978160687 2.607299154652674 2.668482863 0
2.571777029855797 2.612321037076951 2.673589922 1
2.6659300277579416 2.618855641101268 2.68023536 0
2.687942385900994 2.6404519600945564 2.702197976 0
2.681156406942763 2.647820525233347 2.70969152 0
2.698999173634771 2.6519931637558085 2.713934931 0
2.7126522660732544 2.6610411323888283 2.723136363 0
2.734086239504862 2.677746494321589 2.740125066 0
2.6853966898585395 2.6906149110096003 2.753211745 1
2.7521856644125737 2.6928251607749893 2.755459483 0
2.7444176364130364 2.719800108389903 2.782891955 0
1.9633596922697203 2.784328848923095 2.848515173 1
2.206890984959653 2.790882031829411 2.855179505 1
2.8644196110441476 2.817281506096809 2.882026743 0
5.0839303739507535 2.82317978292099 2.888025061 0
2.8885181707088954 2.843693949395364 2

3.00298853822472 2.9746688410965842 3.042083538 0
3.001144936956992 2.985148529213439 3.052740973 0
3.04933964635019 2.992502237343225 3.060219408 0
3.0307361431002127 2.9961851226123497 3.063964759 0
3.004832541148499 3.0027334105427688 3.070624113 0
3.089632977935379 3.032391417558555 3.100785152 0
2.5816597769565526 3.03913934832974 3.107647535 1
3.07178057149818 3.047470749720811 3.116120246 0
2.624801251589555 3.0539135144394174 3.122672287 1
3.1360358996912483 3.071885656408045 3.140949256 0
3.1284229109268624 3.079565086932539 3.148758938 0
3.116084840394816 3.0831684716178405 3.15242344 0
3.1503448964587513 3.098819725369287 3.168340156 0
3.116084840394816 3.104421044911236 3.17403648 0
3.144616477879155 3.1336795882642456 3.20379128 0
3.2577902996668255 3.200990148122816 3.2722435 0
3.2685888225607007 3.2117609860233793 3.283197023 0
3.2764580678847413 3.222549501099856 3.294168523 0
3.2892789289884234 3.2320975862146133 3.303878554 0
3.2197253558448953 3.25530051928956 3.3274

-inf -inf 2.276262564 0
2.235819257792986 2.222652615381758 2.277312301 0
2.2477627012562866 2.228458809036424 2.283216974 0
2.22317833859953 2.2334396466256456 2.288282292 1
2.3005342813348375 2.240335837740508 2.29529545 0
-inf -inf 2.297037982 0
2.261251918026527 2.2532532209039564 2.308431926 0
2.288396837999573 2.2603658027964015 2.315665145 0
2.313478651673776 2.266101441145615 2.321498066 0
2.333376491543635 2.276025631113902 2.331590581 0
2.3257086767284765 2.289909666801518 2.345710105 0
-inf -inf 2.346724745 0
2.340289996297618 2.2959277643364144 2.351830276 0
2.351847849953289 2.311669516142642 2.367839025 0
2.3883186209143004 2.336601530128914 2.393193913 0
2.3104294797731306 2.339903654515807 2.396552045 1
2.360348900243372 2.350745110654418 2.407577384 0
-inf -inf 2.429811873 0
3.3650011357419256 2.3726206015886278 2.429823907 0
2.3379838880887585 2.3800611950489197 2.437390701 1
-inf -inf 2.4427561 0
2.439412483673613 2.4067946398142444 2.464577574 0
2.4465475017389586 2

In [596]:
#Calculate the rounded up number of pixels in the region
lybo6det=(5000/69)
l=mt.floor(lybo6det)
#finds the closest approximate value for the lyB,06 line
q=min(range(len(x)), key=lambda i: abs(x[i]-lybO6*z1))
#The approximate expected values for the region
print(x[q-l],x[q],x[q+l])

4983.1045 5066.4062 5151.1


In [521]:
#the range of concern in km/s
v=5000.0
#speed of light in km/s
c=299792.458
#references the emission redshift from input table
zuse=df.iloc[t]['Zem']
#conveniant for frame conversion
z1=zuse+1
#lyman beta,O6 line
lybO6=1025.7
#observed frame
obslyb=(lybO6*z1)
#converts the km/s value to Angstrom 
lyboffset=(((v/c+1)*lybO6)*(z1))-lybO6*z1
#the lower bound,centre and upper bound of the region where a higher SNR threshold is neccesary
print((lybO6*z1)-lyboffset,(lybO6*z1),(lybO6*z1)+lyboffset)

4981.516168749759 5066.008088973 5150.50000919624


In [591]:


SNRval=5

ZMIN=[]
ZMAX=[]

while wind <SNRval:
        print(SNRval)
        if (x[i]>=lwr) & (x[i]<=upr):
            SNRval=15
            print(SNRval,i)
        else:
            SNRval=5
        wind=np.median(SNR[i:i+33])
        i=i+1
     
        if i==len(SNR)-17:
            A=1
            print('uh oh')
            break   
if A==1:
    ZMIN.append(-np.inf)  
    ZMAX.append(-np.inf)
else:
    ZMIN=np.round(x[i+17]) 
    ZMAX=len(SNR)-87
print(ZMIN,ZMAX,i)        

5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
